## Master Agreement/PSA Report 
* Goal: Make it easy for branch chiefs to know whether a Master Agreement and/or Program Supplement is needed or is going to expire in the next year.
* This will be a monthly report.


In [33]:
import A1_data_prep
import A2_tableau
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Project Sheet
* Column A - Award Year  
* Column B - Project #       
* Column C - Grant Recipient          
* Column D - Project Title 
* Column E - PPNO             
* Column I - Master Agreement Number
* Column J - Master Agreement Expiration Date
* Column K - Project Manager

In [34]:
# Load in sheets
project = A1_data_prep.clean_project()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [35]:
# Subset of cols
project_cols = [
    "project_award_year",
    "project_project_#",
    "project_grant_recipient",
    "project_project_title",
    "project_ppno",
    "project_master_agreement_number",
    "project_master_agreement_expiration_date",
    "project_project_manager",
]

In [36]:
# Subset df to only the columns requested
project2 = project[project_cols]

In [37]:
project2.project_master_agreement_expiration_date = project2.project_master_agreement_expiration_date.apply(pd.to_datetime, errors="coerce")

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


#### Criteria
* If the master agreement field (column I Master Agreement Number, tab 1) is empty, the information should appear on the report
* If there is less than a year until the Master Agreement Expiration Date, the information should appear on the report

In [38]:
project_cond1= (project2.project_master_agreement_expiration_date.dt.year == 2023)
project_cond2 = (project2.project_master_agreement_number.isin(["None","Pending"]))

In [39]:
# Filter based on criteria above
project3 = (project2[
    (project_cond1 | project_cond2)
]).reset_index(drop = True)

In [40]:
# Tag projects with the appropriate comment
def ma_comments(row):

    if (row["project_master_agreement_number"] == "None")|(row["project_master_agreement_number"] == "Pending"):
        return "None/Pending Master Agreement Number"
    else:
        return "<1 a year until Master Agreement Expiration Date"

In [41]:
project3["Comments"] = project3.apply(ma_comments, axis=1)

### Allocation Sheet
* Column F - Project ID
* Column E - EA
* Column I - Phase 
* Column J - Allocation Amount
* Column T - Allocation Date 
* Column V - PSA #
* Column Q: Allocation Amendment
* Column AB - Date Branch Chief Receives PSA       
* Column AC - Date Regional Coordinator Receives PSA      
* Column AD - Date OC Receives PSA          
* Column AE - Date OPM Receives PSA      
* Column AF - Date Legal Receives PSA      
* Column AG - Date Returned to PM           
* Column AH - Date PSA Sent to Local Agency         
* Column AI - Date PSA Approved by Local Agency 
* Column AJ - Date Signed by DRMT
* Column AK – PSA Expiry Date


In [42]:
alloc = A1_data_prep.clean_allocation()

In [43]:
# Subset of cols
alloc_cols = [
    "allocation_ppno",
    "allocation_project_id",
    "allocation_ea",
    "allocation_phase",
    "allocation_allocation_amount",
    "allocation_allocation_date",
    "allocation_psa_#",
    "allocation_date_branch_chief_receives_psa",
    "allocation_date_regional_coordinator_receives_psa",
    "allocation_date_oc_receives_psa",
    "allocation_date_opm_receives_psa",
    "allocation_date_legal_receives_psa",
    "allocation_date_returned_to_pm",
    "allocation_date_psa_sent_to_local_agency",
    "allocation_date_psa_approved_by_local_agency",
    "allocation_ctc_allocation_amendment",
    "allocation_date_signed_by_drmt",
    "allocation_psa_expiry_date",
]

In [44]:
alloc2 = alloc[alloc_cols]

In [66]:
alloc2.isna().sum()/len(alloc2)

allocation_ppno                                     0.00
allocation_project_id                               0.00
allocation_ea                                       0.00
allocation_phase                                    0.00
allocation_allocation_amount                        0.00
allocation_allocation_date                          0.25
allocation_psa_#                                    0.00
allocation_date_branch_chief_receives_psa           0.97
allocation_date_regional_coordinator_receives_psa   1.00
allocation_date_oc_receives_psa                     0.93
allocation_date_opm_receives_psa                    0.67
allocation_date_legal_receives_psa                  0.66
allocation_date_returned_to_pm                      0.67
allocation_date_psa_sent_to_local_agency            0.00
allocation_date_psa_approved_by_local_agency        0.88
allocation_ctc_allocation_amendment                 0.00
allocation_date_signed_by_drmt                      0.39
allocation_psa_expiry_date     

In [71]:
# alloc2[['allocation_ctc_allocation_amendment']]

#### Criteria
* If there is a date in the “Allocation Date” field (column T) and no date in the “Date Signed by DRMT” (column AJ), then the information should be on the report
* If there is 6 months or less until the expiry date (column AK), then the information would be on the report


In [57]:
alloc_cond1 = ((alloc2.allocation_allocation_date.notna()) & (alloc2.allocation_date_signed_by_drmt.isna())) 

In [58]:
alloc_cond2 = (alloc2.allocation_psa_expiry_date.dt.date.astype(str) > '2022-12-31') & (alloc2.allocation_psa_expiry_date.dt.date.astype(str) < '2023-06-12')

In [59]:
alloc3 = (alloc2[alloc_cond1 |(alloc_cond2)]).reset_index(drop = True)

In [60]:
len(alloc3)

54

In [62]:
# alloc3[['allocation_allocation_date','allocation_date_signed_by_drmt','allocation_psa_expiry_date']]

* If there is new information in column Q (allocation_ctc_allocation_amendment) and no change in column, then the information should be on the report

In [80]:
def previous_allocation():
    df = df = to_snakecase(
        pd.read_excel(f"{A1_data_prep.GCS_FILE_PATH}fake_allocation.xlsx", sheet_name="Agreement Allocations")
    )

    """
    Some rows are not completely filled: drop them based on whether or not some
    cols are populated.
    """
    df = df.dropna(subset=["award_year", "grant_recipient", "ppno"])

    # Correcting string to 0
    df["expended_amount"] = (
        df["expended_amount"].replace({"Deallocation": 0}).astype("int64")
    )

    # Fill in NA based on data type
    df = df.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))

    # Coerce dates to datetime
    # df[date_columns] = df[date_columns].apply(pd.to_datetime, errors="coerce")

    # Clean organization name/de duplicate
    df = A1_data_prep.organization_cleaning(df, "grant_recipient")
    
    # Add prefix
    df = df.add_prefix("allocation_")
    
    df = df[alloc_cols]
    df = df.add_prefix("fake_")
    return df


In [81]:
alloc_fake_test = previous_allocation()